# Evaluation of the operator constructed in openfermion on the state.


In [21]:
from pyqtorch.core.operation import Z, Y, X, RX
from pyqtorch.embedding import SingleLayerEncoding
from pyqtorch.ansatz import AlternateLayerAnsatz
from pyqtorch.core.circuit import QuantumCircuit
from pyqtorch.core.measurement import total_magnetization

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

import math
import networkx as nx
import numpy as np
import matplotlib as mpl

mpl.rcParams["figure.figsize"] = [5, 5]

In [22]:
def generate_all_possible_local_terms(n_qubits, initial_list, ops = ["I", "Z"]):
    """Generate all possible local terms."""
    new_pauli_op_list = []
    for el in initial_list:
        for op1 in ops:
            new_pauli_op_list.append(el+op1)
    if len(new_pauli_op_list[-1]) == n_qubits:
        return new_pauli_op_list
    else:
        return generate_all_possible_local_terms(n_qubits, new_pauli_op_list, ops=ops)


def filter_k_local_terms(list_pauli_terms, locality, ops = ["Z"]):
    """Filter k local terms."""
    terms = []
    for term in list_pauli_terms:
        k = 0
        for char in term:
            if char in ops:
                k+=1
        if k == locality:
            terms.append(term)
    return terms

In [27]:
n_qubits = 2

# generate all pauli terms and divide into some sets

list_all_pauli_terms = generate_all_possible_local_terms(n_qubits, [""])
print(list_all_pauli_terms)
print("---------------------------------------------------")
one_local_terms = filter_k_local_terms(list_all_pauli_terms, 1)
print(one_local_terms)
two_local_terms = filter_k_local_terms(list_all_pauli_terms, 2)
print(two_local_terms)
three_local_terms = filter_k_local_terms(list_all_pauli_terms, 3)
print(three_local_terms)
four_local_terms = filter_k_local_terms(list_all_pauli_terms, 4)
print(four_local_terms)
five_local_terms = filter_k_local_terms(list_all_pauli_terms, 5)
print(five_local_terms)
print("---------------------------------------------------")
list_all_pauli_X_terms = generate_all_possible_local_terms(n_qubits, [""], ops = ["I","X"])
print(list_all_pauli_X_terms)
one_local_X_terms = filter_k_local_terms(list_all_pauli_X_terms, 1, ops = ["X"])
print(one_local_X_terms)

hamiltonian_pauli_terms = list_all_pauli_terms

['II', 'IZ', 'ZI', 'ZZ']
---------------------------------------------------
['IZ', 'ZI']
['ZZ']
[]
[]
[]
---------------------------------------------------
['II', 'IX', 'XI', 'XX']
['IX', 'XI']


In [28]:
from openfermion.ops import QubitOperator

# TODO: I need to convert to openfermion format.


def convert_pauli_to_openfermion_stringformat(pauli_str):
    openfermion_pauli_str = ""
    for qubit, term in enumerate(pauli_str):
        if term != "I":
            openfermion_pauli_str += f"{term}{qubit} "
    return openfermion_pauli_str.rstrip()


openfermion_pauli_str = convert_pauli_to_openfermion_stringformat("IXIZIY")
print(openfermion_pauli_str)


# Convert from string to this way.
operator = QubitOperator(openfermion_pauli_str, 3.17)
print(operator)

X1 Z3 Y5
3.17 [X1 Z3 Y5]


In [32]:
hamiltonian_coefficients = nn.Parameter(
    torch.ones(
        len(hamiltonian_pauli_terms),
    )
)
print(hamiltonian_coefficients)

Parameter containing:
tensor([1., 1., 1., 1.], requires_grad=True)


In [ ]:
# TODO: this is original function
def measure_openfermion(state, operator, N_qubits, batch_size):
    new_state = torch.zeros_like(state)

    for op, coef in operator.terms.items():
        for qubit, pauli in op:
            state_bis = qubit_operators[pauli](state, [qubit], N_qubits)
            new_state += state_bis * coef

    state = state.reshape((2**N_qubits, batch_size))
    new_state = new_state.reshape((2**N_qubits, batch_size))

    return torch.real(
        torch.sum(torch.conj(state) * new_state, axis=0))

In [34]:
# Generate state with different batches.
batch_size = 1
state = QuantumCircuit(n_qubits).init_state(batch_size)
# Reshaping was only necessary for the expectation value.
# state = state.reshape((2**n_qubits, batch_size))
print(state)
print(state.shape)


new_state = torch.zeros_like(state)

qubit_operators = {'X': X, 'Y': Y, 'Z': Z}

for op, coef in operator.terms.items():
    for qubit, pauli in op:
        state_bis = qubit_operators[pauli](state, [qubit], n_qubits)
        new_state += state_bis * coef

state = state.reshape((2**N_qubits, batch_size))
new_state = new_state.reshape((2**N_qubits, batch_size))

expectation_value =  torch.real(
    torch.sum(torch.conj(state) * new_state, axis=0))

tensor([[[1.+0.j],
         [0.+0.j]],

        [[0.+0.j],
         [0.+0.j]]], dtype=torch.complex128)
torch.Size([2, 2, 1])


NameError: name 'N_qubits' is not defined

***
# Outcome:

- ...
